In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import time
from auto_tsmo import full_method
from filter_big import bFilter, bPostProcessing
from filter_med import mFilter, mPostProcessing
from filter_small import sFilter, sPostProcessing
from drawing import showRect, showRects
from feature_extraction import getFeatures, getFeaturesVector

video_path = r'C:\Users\admin\Google\HumanDetection\Data\video.mp4'

big_dataset = r"C:\Users\admin\Google\HumanDetection\Final\big_dataset"
med_dataset = r"C:\Users\admin\Google\HumanDetection\Final\med_dataset"
small_dataset = r"C:\Users\admin\Google\HumanDetection\Final\small_dataset"

In [4]:
cap = cv2.VideoCapture(video_path)

start = time.time()

first_frame = 1000
curr = 0

while(cap.isOpened()):
    curr += 1
    
    ret, frame = cap.read()
    
    if (curr < first_frame):
        start = time.time()
        continue
    
    frame = frame[10:-10,40:-40]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    thresholds, masks = full_method(gray)
    
    contours = sFilter(thresholds, masks)
    rects = sPostProcessing(contours)
    showRects(frame, rects)
    
    contours = mFilter(thresholds, masks)
    rects = mPostProcessing(contours)
    showRects(frame, rects)
    
    contours = bFilter(thresholds, masks)
    rects = bPostProcessing(contours)
    showRects(frame, rects)

        
    cv2.imshow('frame',frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
end = time.time()

print("Average FPS:", (curr-first_frame)/(end-start))

cap.release()
cv2.destroyAllWindows()

Average FPS: 17.766228434942484


In [55]:
import numpy as np
import cv2

falses = []
trues = []

falses.append(frame)
trues.append(frame)

cap = cv2.VideoCapture(video_path)

end = 8000
curr = 0


while(cap.isOpened()):
    curr += 1
    
    ret, frame = cap.read()
    
    if (curr > end):
        break
        
    if (curr % 25 != 0):
        continue
        
    frame = frame[10:-10,40:-40]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    thresholds, masks = full_method(gray)
    
    dataset = big_dataset
    contours = bFilter(thresholds, masks)
    rects = bPostProcessing(contours)
    
  #  print(len(rects))
    
    for rect in rects:
        frame_ = frame.copy()
        x,y,w,h = rect
        
        i,j,step_width,step_height = x,y,w,h
        
        cv2.rectangle(frame_, (x, y), (x+w, y+h), (0, 0, 255), 2)
        cv2.imshow('roi', frame_)
        
        if cv2.waitKey(1000000) & 0xFF == ord('n'):
            print("false")
            if not any([np.array_equal(frame[j:j+step_height,i:i+step_width],f) for f in falses]): 
                falses.append(frame[j:j+step_height,i:i+step_width])
                
                cv2.imwrite(dataset + '/falses/frame' + str(curr) + '_' + str(i) + '_' + str(j) + '.jpg', 
                        frame[j:j+step_height,i:i+step_width])
            else:
                print("NOFALSE")
            print(len(falses))
        else:
            print("true")
            if not any([np.array_equal(frame[j:j+step_height,i:i+step_width],t) for t in trues]): 
        
                trues.append(frame[j:j+step_height,i:i+step_width])
                cv2.imwrite(dataset + '/trues/frame' + str(curr) + '_' + str(i) + '_' + str(j) + '.jpg', 
                        frame[j:j+step_height,i:i+step_width])
            else:
                print("NOTRUE")
            print(len(trues))
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

true
2
true
3
true
4
true
5
true
6
true
7
true
8
true
9
true
10
true
11
true
12
true
13
true
14
true
15
true
16
true
17
true
18
false
2
true
19
true
20
true
21
true
22
true
23
true
24
true
25
true
26
false
3
false
4
true
27
true
28
true
29
false
5
true
30
true
31
true
32
false
6
true
33
true
34
true
35
true
36
false
7
true
37
true
38
true
39
true
40
true
41
true
42
true
43
true
44
true
45
true
46
true
47
true
48
true
49
true
50
true
51
true
52
true
53
true
54
true
55
true
56
true
57
true
58
true
59
true
60
true
61
true
62
true
63
true
64
true
65
true
66
true
67
true
68
true
69
true
70
true
71
true
72
true
73
true
74
true
75
true
76
true
77
true
78
true
79
true
80
true
81
true
82
true
83
true
84
true
85
true
86
true
87
true
88
true
89
true
90
true
91
true
92
true
93
true
94
true
95
true
96
true
97
false
8
true
98
false
9
true
99
false
10
false
11
true
100
false
12
true
101
false
13
false
14
false
15
false
16
false
17
false
18
false
19
true
102
false
20
true
103
false
21
true
104
false
2

In [35]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import glob

dataset = big_dataset

true = [cv2.imread(file,0) for file in glob.glob(dataset + "/trues/*.jpg")]
false = [cv2.imread(file,0) for file in glob.glob(dataset + "/falses/*.jpg")]
np.random.shuffle(true)
np.random.shuffle(false)

print(len(true), len(false))

true_train, true_test = train_test_split(true[:-1], test_size=0.33, random_state=42, shuffle = False)
false_train, false_test = train_test_split(false[:-1], test_size=0.33, random_state=42, shuffle = False)

print(len(true_train), len(false_train))

train = np.append(true_train, false_train)
train_labels = np.append(np.ones(len(true_train)), np.zeros(len(false_train)))
train, train_labels = shuffle(train, train_labels, random_state=42)

test = np.append(true_test, false_test)
test_labels = np.append(np.ones(len(true_test)), np.zeros(len(false_test)))
test, test_labels = shuffle(test, test_labels, random_state=42)

113 77
75 50


In [36]:
print("LBP: ", getFeatures(train[0], add_HOG = False, add_LBP = True).shape)
print("HOG: ", getFeatures(train[0], add_HOG = True, add_LBP = False).shape)
print("LBP&HOG: ", getFeatures(train[0], add_HOG = True, add_LBP = True).shape)

LBP:  (1, 3776)
HOG:  (1, 3780)
LBP&HOG:  (1, 7556)


In [50]:
add_LBP = True
add_HOG = True

train_features = getFeaturesVector(train, add_HOG, add_LBP)
test_features = getFeaturesVector(test, add_HOG, add_LBP)
print(train_features.shape, test_features.shape)

(125, 7556) (63, 7556)


In [51]:
true_feats = getFeaturesVector(true_test, add_HOG, add_LBP)
false_feats = getFeaturesVector(false_test, add_HOG, add_LBP)

In [52]:
from sklearn.svm import LinearSVC
model = LinearSVC(C=1,random_state=42, max_iter=100000)                         
print(model)
model.fit(train_features, train_labels)
print(model.score(test_features, test_labels))
#dump(model, 'gbc_1.joblib') 

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=100000,
          multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
          verbose=0)
0.9682539682539683


In [53]:
print("FN",(1 - model.score(true_feats, np.ones(len(true_feats))))*100)
print("FP",(1 - model.score(false_feats, np.zeros(len(false_feats))))*100)

FN 2.7027027027026973
FP 3.8461538461538436


In [49]:
print(model.score(true_feats, np.ones(len(true_feats))))
print(model.score(false_feats, np.zeros(len(false_feats))))

0.972972972972973
0.9615384615384616


In [80]:
from joblib import dump, load
dump(model, 'svm_big_hog_lbp.joblib') 

['svm_big_hog_lbp.joblib']

In [20]:
model = load('svm_small_hog_lbp.joblib') 